In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam
from keras.layers import Dropout
from sklearn.metrics import classification_report, accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
url = "/kaggle/input/pima-indians-diabetes-database/diabetes.csv"
names = ['n_pregnant', 'glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)',
        'BMI', 'pedigree_function', 'age', 'class']
df = pd.read_csv(url, names = names)
df.head()

In [ ]:
df.describe()

In [ ]:
columns = ['glucose_concentration', 'blood_pressuer (mm Hg)', 'skin_thickness (mm)', 'serum_insulin (mu U/ml)', 'BMI']
for col in columns:
    df[col].replace(0, np.NaN, inplace=True)
df.describe()

In [ ]:
df.dropna(inplace=True) 
df.describe()

In [ ]:
dataset = df.values  # Convert dataframe to numpy array
dataset.shape

In [ ]:
# Split into input (X) and output (Y)
X = dataset[1:,0:8]   # Array slicing, select all rows and columns from 0 to 7
Y = dataset[1:,8]
print(X.shape)
print(Y.shape)
print(Y[:5])

In [ ]:
# Standardize the data
scaler = StandardScaler().fit(X)
X_standardized = scaler.transform(X)
data = pd.DataFrame(X_standardized)
data.describe()

In [ ]:
# Building the Keras model
def create_model():
    # create sequential model
    model = Sequential()    
    model.add(Dense(8, input_dim = 8, kernel_initializer = 'normal', activation = 'relu')) #rectifier(relu) activation function
    model.add(Dense(4, input_dim = 8, kernel_initializer = 'normal', activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid')) #sigmoid function in output layer

    #compile the model
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy']) # metrics for classification problem: classification accuracy
    return model

model = create_model()
model.summary()

Optimizing network by tuning the hyperparameters

In [ ]:


# Performing a grid search for the optimal batch size and number of epochs
seed = 6  # Define a random seed
np.random.seed(seed)

# Start defining the model
def create_model():
    
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr = 0.01)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

#Create the model
model = KerasClassifier(build_fn = create_model, verbose = 1)

# define the grid search parameters
batch_size = [10,20,40]
epochs = [10, 50, 100]

# make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size, epochs = epochs)

# build and fit the GridSearchCV- exhaustive search over specified parameter values for an estimator
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state = seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

In [ ]:
# Summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
std = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, std, params):
    print('{0} ({1} with: {2}'.format(mean, stdev, param))

In [ ]:
# Do a grid search for learning rate and dropout rate
# Reducing overfitting using dropout regularization
seed = 6
np.random.seed(seed)

# defining the model
def create_model(learn_rate, dropout_rate):    
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate = learn_rate, dropout_rate = dropout_rate)

# build and fit the GridSearchCV- exhaustive search over specified parameter values for an estimator
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state = seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

In [ ]:
# Summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
std = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, std, params):
    print('{0} ({1} with: {2}'.format(mean, stdev, param))

In [ ]:
# Do a grid search to optimize kernel initialization and activation functions
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation=activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    # We will modify the learning rate of the Adam optimizer to 0.001, as this is the best value that we found.
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# Define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV- exhaustive search over specified parameter values for an estimator
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state = seed), verbose = 10)
grid_results = grid.fit(X_standardized, Y)

In [ ]:
# Summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
std = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, std, params):
    print('{0} ({1} with: {2}'.format(mean, stdev, param))

In [ ]:
# Do a grid search to find the optimal number of neurons in each hidden layer
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer='uniform', activation='linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# Create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# Define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1=neuron1, neuron2=neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state = seed), refit=True, verbose = 10)
grid_results = grid.fit(X_standardized, Y)

In [ ]:
# Summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
std = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, std, params):
    print('{0} ({1} with: {2}'.format(mean, stdev, param))

In [ ]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(X_standardized)
print(y_pred.shape)
print(y_pred[:5])

print(accuracy_score(Y,y_pred))
print(classification_report(Y,y_pred))

In [ ]:
#prediction for any example
example = df.iloc[5]
print(example)
print("Actual class",Y[4])
prediction = grid.predict(X_standardized[4].reshape(1,-1))
print("Predicted class", prediction)